In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import data 

In [ ]:
pathimage="/kaggle/input/automatic-classification-of-fetal-ultrasound-brain/data/data/"
pathcsv_train="/kaggle/input/automatic-classification-of-fetal-ultrasound-brain/train.csv"
pathcsv_test="/kaggle/input/automatic-classification-of-fetal-ultrasound-brain/test.csv"

In [ ]:
train_csv=pd.read_csv(pathcsv_train)
test_csv=pd.read_csv(pathcsv_test)

In [ ]:
train_csv['image_name']=train_csv['image_name']+".png"
test_csv['image_name']=test_csv['image_name']+".png"

In [ ]:
train_csv.head()

In [ ]:
from PIL import Image
from matplotlib.pyplot import imshow
import cv2
import matplotlib.pyplot as plt

### Let's see some image

In [ ]:
image = cv2.imread('/kaggle/input/automatic-classification-of-fetal-ultrasound-brain/data/data/Patient00779_Plane3_3_of_3.png')
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Original Image')
plt.show()

In [ ]:
import tensorflow as tf

### let's do some data augmentation on the images 

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.20)
test_datagen = ImageDataGenerator(rescale=1 / 255.0)

In [ ]:
batch_size = 8
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_csv,
    directory=pathimage,
    x_col="image_name",
    y_col="class",
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode="categorical",
    subset='training',
    shuffle=True,
    seed=42
)


   

In [ ]:
valid_generator = train_datagen.flow_from_dataframe(
    dataframe=train_csv,
    directory=pathimage,
    x_col="image_name",
    y_col="class",
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode="categorical",
    subset='validation',
    shuffle=True,
    seed=42
)

In [ ]:
test_generator = test_datagen.flow_from_dataframe( dataframe=test_csv,
   directory=pathimage,
    x_col="image_name",
    target_size=(100, 100),
    batch_size=1,
    class_mode=None,
    shuffle=False)

### Then we build our model

In [ ]:
def prepare_model():
    model = Sequential()
    model.add(Conv2D(input_shape=(100,100,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=4, activation="softmax"))
    return model

In [ ]:
model = prepare_model()
model.compile(optimizer="adam", loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("./vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit_generator(steps_per_epoch=20,generator=train_generator, validation_data= valid_generator, validation_steps=10,epochs=20,callbacks=[checkpoint,early])

In [ ]:
#comment below line if you run it second time
model.save("./vgg16_1.h5")

### Then we make prediction

In [ ]:
from keras import models    
model = models.load_model("./vgg16_1.h5", compile = False)

In [ ]:
test_csv=pd.read_csv(pathcsv_test)

In [ ]:
output = model.predict(test_generator)

In [ ]:
test_csv['class'] = np.argmax(output, axis=-1) 

In [ ]:
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_csv['class'] = test_csv['class'].replace(label_map)
print(label_map, "\n")

# See the output of mapped labels
print(test_csv['class'][0:4], "\n")

In [ ]:
test_csv

In [ ]:
test_csv.to_csv("./mysubmission3.csv",index=False)